## Stack Overflow Tag Analysis

The Stack Overflow dataset is a seminal dataset used by tech market researchers, because it is open, comprehensive, and behavioral in nature. It was first made available for public consumption in 2010, and encompasses not just questions and answer posts from the developer community but also technology tags associated with each post. As a result, these tags act as a good market proxy of user adoption, interest or disterest from a behavioral perspective: the more posts with a certain tag, the more indicative interest of that certain tag. The time series nature of this dataset also make it possible to gauge the waxing and waning of a tag, while also indicating changing affiliation among tags. Finally, additional metadata about users, badge contests and an annual developer survey provide additional context to help make business/actionable decisions, leading to measurable outcomes.

Thus, for my capstone project I would like to analyze technology tags in the Stack Overflow dataset in order to better understand technology adoption curves and affiliations among technologies, depending on certain cohorts of developers. My team and I at Microsoft perfomed a similar analysis in 2016, where we found that OSS developers gravitated towards AWS and Google, as there were very few OSS offerings on Azure. We also realized that Microsoft had not reached out to OSS developers proactively, even with the limited offers. This 'wake up call' generated massive investment in development and marketing of OSS technologies on the Azure cloud/platform.

Five years later, Microsoft would like to know if their efforts have paid off. They have seen fantastic growth in their OSS services and technologies, but are they keeping up with the market? Have they been able bring new developers onto their platform, or is this adoption coming from existing developers who have become 'polyglots'? And have the 'islands of technologies' that used to center around just AWS and Google begun to diffuse and include Azure? ...or is Azure still an island unto itself?

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
sns.set()
%matplotlib inline

#preprocessing
from sklearn.preprocessing import StandardScaler, Normalizer, MinMaxScaler
from scipy import stats

# pipelines
from sklearn.pipeline import Pipeline

# Regression
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.linear_model import Ridge
from sklearn.metrics import mean_squared_error

# NLP transformers
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer

# classifiers you can use
from sklearn.dummy import DummyClassifier
from sklearn.svm import SVC
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier

# model selection bits
from sklearn.model_selection import GridSearchCV, train_test_split, cross_val_score, ShuffleSplit, RandomizedSearchCV
from sklearn.model_selection import StratifiedShuffleSplit, StratifiedKFold, KFold
from sklearn.model_selection import learning_curve, validation_curve

# evaluation
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, classification_report, fbeta_score

In [2]:
posts = pd.read_csv("/Users/bens_mac/Documents/CodingNomads/SO_posts_BIG.csv")
users = pd.read_csv("/Users/bens_mac/Documents/CodingNomads/SO_users_BIG.csv")

In [3]:
posts.head()

,id,creation_date,last_activity_date,owner_user_id,post_type_id,tags,view_count
0,66138537,2021-02-10 14:11:57.947 UTC,2021-02-10 14:11:57.947 UTC,8384006.0,1,dataexplorer,2
1,66229417,2021-02-16 17:40:44.097 UTC,2021-02-16 17:40:44.097 UTC,12549160.0,1,rstudio-server,2
2,66288134,2021-02-20 04:49:09.76 UTC,2021-02-20 04:49:09.76 UTC,15246800.0,1,routes,2
3,66293452,2021-02-20 15:43:15.133 UTC,2021-02-20 15:43:15.133 UTC,7822211.0,1,angular-dynamic-components,2
4,66361333,2021-02-25 01:56:26.023 UTC,2021-02-25 01:56:26.023 UTC,2713214.0,1,amazon-eks|elastic-network-interface,2


In [4]:
users.head()

,id,display_name,age,creation_date,last_access_date,location,reputation
0,14712167,nicom,NaN,2020-11-26 09:26:50.507 UTC,2021-02-27 13:20:10.693 UTC,NaN,19
1,14717603,Donkey,NaN,2020-11-27 05:56:10.18 UTC,2021-02-18 10:38:39.56 UTC,NaN,43
2,14785218,redshorts17,NaN,2020-12-08 07:42:10.147 UTC,2021-01-08 13:08:27.023 UTC,NaN,89
3,14808842,PerekatovSergey,NaN,2020-12-11 16:09:17.147 UTC,2021-02-26 14:26:46.983 UTC,Russia,26
4,14916620,leung2,NaN,2020-12-31 03:02:17.93 UTC,2021-02-14 11:27:27.557 UTC,NaN,33


In [5]:
posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10445772 entries, 0 to 10445771
Data columns (total 7 columns):
 #   Column              Dtype  
---  ------              -----  
 0   id                  int64  
 1   creation_date       object 
 2   last_activity_date  object 
 3   owner_user_id       float64
 4   post_type_id        int64  
 5   tags                object 
 6   view_count          int64  
dtypes: float64(1), int64(3), object(3)
memory usage: 557.9+ MB


In [6]:
users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 14080580 entries, 0 to 14080579
Data columns (total 7 columns):
 #   Column            Dtype  
---  ------            -----  
 0   id                int64  
 1   display_name      object 
 2   age               float64
 3   creation_date     object 
 4   last_access_date  object 
 5   location          object 
 6   reputation        int64  
dtypes: float64(1), int64(2), object(4)
memory usage: 752.0+ MB


In [7]:
posts.dropna()

,id,creation_date,last_activity_date,owner_user_id,post_type_id,tags,view_count
0,66138537,2021-02-10 14:11:57.947 UTC,2021-02-10 14:11:57.947 UTC,8384006.0,1,dataexplorer,2
1,66229417,2021-02-16 17:40:44.097 UTC,2021-02-16 17:40:44.097 UTC,12549160.0,1,rstudio-server,2
2,66288134,2021-02-20 04:49:09.76 UTC,2021-02-20 04:49:09.76 UTC,15246800.0,1,routes,2
3,66293452,2021-02-20 15:43:15.133 UTC,2021-02-20 15:43:15.133 UTC,7822211.0,1,angular-dynamic-components,2
4,66361333,2021-02-25 01:56:26.023 UTC,2021-02-25 01:56:26.023 UTC,2713214.0,1,amazon-eks|elastic-network-interface,2
...,...,...,...,...,...,...,...
10445766,42686301,2017-03-09 03:48:06.533 UTC,2017-03-10 11:30:01.587 UTC,2410131.0,1,mysql,95
10445767,42670128,2017-03-08 11:29:40.897 UTC,2017-03-08 20:26:48.12 UTC,1958365.0,1,c#|sql-server|asp.net-mvc|entity-framework,1452
10445769,42905797,2017-03-20 14:09:22.073 UTC,2017-03-20 17:11:44.09 UTC,7709628.0,1,php|mysql|select|pdo,80
10445770,43037016,2017-03-27 02:25:18.11 UTC,2017-03-27 16:14:33.847 UTC,7771458.0,1,php|html|mysql|xampp,724


In [8]:
posts['tags'] = posts['tags'].str.split("|")
posts['creation_date'] = [x[:4] for x in posts['creation_date']]
posts['last_activity_date'] = [x[:4] for x in posts['last_activity_date']]

In [9]:
users['creation_date'] = [x[:4] for x in users['creation_date']]
users['last_access_date'] = [x[:4] for x in users['last_access_date']]

In [10]:
posts['creation_date'] = posts['creation_date'].astype(str).astype(float)
posts['last_activity_date'] = posts['last_activity_date'].astype(str).astype(float)

In [11]:
posts.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10445772 entries, 0 to 10445771
Data columns (total 7 columns):
 #   Column              Dtype  
---  ------              -----  
 0   id                  int64  
 1   creation_date       float64
 2   last_activity_date  float64
 3   owner_user_id       float64
 4   post_type_id        int64  
 5   tags                object 
 6   view_count          int64  
dtypes: float64(3), int64(3), object(1)
memory usage: 557.9+ MB


In [12]:
posts_proto = posts.sample(n=10000)

In [13]:
posts_proto.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 10000 entries, 9472402 to 9637119
Data columns (total 7 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   id                  10000 non-null  int64  
 1   creation_date       10000 non-null  float64
 2   last_activity_date  10000 non-null  float64
 3   owner_user_id       9815 non-null   float64
 4   post_type_id        10000 non-null  int64  
 5   tags                10000 non-null  object 
 6   view_count          10000 non-null  int64  
dtypes: float64(3), int64(3), object(1)
memory usage: 625.0+ KB


In [14]:
posts_proto = posts_proto.dropna()

In [15]:
all_tags_proto = posts_proto['tags']

In [16]:
all_tags_proto[0:15]

9472402                      [maven-2, maven-assembly-plugin]
5683859     [entity-framework, ef-code-first, dbcontext, e...
5956129                         [java, jsp, tomcat, netbeans]
8999094      [asp.net-mvc, nhibernate, ninject, multi-tenant]
2099189            [mvvm, prism, delegatecommand, canexecute]
310998      [ruby, ruby-on-rails-4, state-machine, acts-as...
5364289                [java, android, google-maps, firebase]
6956698                     [python, button, kivy, textfield]
1040239                          [php, mysql, database, echo]
10421378                       [c#, asp.net-mvc, inheritance]
8183008     [google-cloud-storage, google-cloud-platform, ...
266034      [java, android, firebase, firebase-storage, pi...
3938547                    [sql-server, azure, remote-access]
6422838           [swift, openwhisk, serverless-architecture]
9148042                 [c#, keyboard-hook, setwindowshookex]
Name: tags, dtype: object

In [36]:
#all tags of postgres + ...
#all tags of mysql + ...
#all tags of mariadb + ...
#all tags of sql-server + ...

#tokenize list of tags and their combinations

postgres = []
element = "postgres"
for tag_list in all_tags_proto:
    for tag in tag_list:
        if element in tag:
            postgres.append(tag_list)
postgres = [' '.join([str(c) for c in lst]) for lst in postgres]

            
#creating X
postgres_indicators = []
for label in postgres:
    if label != "postgresql":
        postgres_indicators.append(label)
postgres_indicators = np.unique(postgres_indicators)

#creating y
postgres_label = []
for i in range(len(postgres_indicators)):
    postgres_label.append("postgres")

print(len(postgres)) #postgres could come up as a single tag in a post.
print(len(postgres_indicators))
print(len(postgres_label))

65
55
55
['postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres', 'postgres']


In [37]:
mysql = []
element = "mysql"
for tag_list in all_tags_proto:
        if element in tag_list:
            mysql.append(tag_list)
mysql = [' '.join([str(c) for c in lst]) for lst in mysql]

#creating X
mysql_indicators = []
for label in mysql:
    if label != "mysql":
        mysql_indicators.append(label)
mysql_indicators = np.unique(mysql_indicators)

#creating y
mysql_label = []
for i in range(len(mysql_indicators)):
    mysql_label.append("mysql")

print(len(mysql)) #mysql could come up as a single tag in a post.
print(len(mysql_indicators))
print(len(mysql_label))

270
195
195


In [38]:
mariadb = []
element = "mariadb"
for tag_list in all_tags_proto:
        if element in tag_list:
            mariadb.append(tag_list)
mariadb = [' '.join([str(c) for c in lst]) for lst in mariadb]

#creating X
mariadb_indicators = []
for label in mariadb:
    if label != "mariadb":
        mariadb_indicators.append(label)
mariadb_indicators = np.unique(mariadb_indicators)

#creating y
mariadb_label = []
for i in range(len(mariadb_indicators)):
    mariadb_label.append("mariadb")

print(len(mariadb)) #mariadb could come up as a single tag in a post.
print(len(mariadb_indicators))
print(len(mariadb_label))
print(mariadb)


6
6
6
['mysql xampp mariadb', 'mysql mariadb', 'node.js mariadb', 'mysql sockets mariadb', 'mariadb command-line-interface', 'sql indexing mariadb']


In [39]:
sql_server = []
element = "sql-server"
for tag_list in all_tags_proto:
        if element in tag_list:
            sql_server.append(tag_list)
sql_server = [' '.join([str(c) for c in lst]) for lst in sql_server]

#creating X
sql_server_indicators = []
for label in sql_server:
    if label != "sql-server":
        sql_server_indicators.append(label)
sql_server_indicators = np.unique(sql_server_indicators)

#creating y
sql_server_label = []
for i in range(len(sql_server_indicators)):
    sql_server_label.append("sql_server")

print(len(sql_server)) #sql-server could come up as a single tag in a post.
print(len(sql_server_indicators))
print(len(sql_server_label))
type(sql_server_indicators)

136
94
94


numpy.ndarray

In [44]:
X = np.concatenate((postgres_indicators, mysql_indicators, mariadb_indicators, sql_server_indicators), axis = 0 )
y = np.concatenate((postgres_label, mysql_label, mariadb_label, sql_server_label), axis = 0)

X_train, X_test, y_train, y_test = train_test_split(X, y, stratify = y, test_size = 0.3, random_state = 20)


#transform
countVec = CountVectorizer(ngram_range=(1,3))
countVec.fit(X_train)

# true_k = 2
# model = KMeans(n_clusters=true_k, init='k-means++', max_iter=100, n_init=1)
# model.fit(X)

# print("Top terms per cluster:")
# order_centroids = model.cluster_centers_.argsort()[:, ::-1]
# terms = vectorizer.get_feature_names()
# for i in range(true_k):
#     print("Cluster %d:" % i),
#     for ind in order_centroids[i, :10]:
#         print(' %s' % terms[ind]),
#     print

CountVectorizer(ngram_range=(1, 3))

In [45]:
len(countVec.vocabulary_)

1315

In [46]:
X_train_vec = countVec.transform(X_train)

In [47]:
#Baseline Model

dummy = DummyClassifier(strategy='most_frequent')
scores = cross_val_score(dummy, X_train, y_train, cv=3, scoring = "accuracy") 
print (f"{scores.mean()} is the mean score")
scores

0.5591689250225836 is the mean score


array([0.56097561, 0.56097561, 0.55555556])

In [49]:
# create the pipeline
dt_pipe = Pipeline([('vect', CountVectorizer(ngram_range=(1,1))),('clf', DecisionTreeClassifier())])

# pass the pipeline as if it was the classifier into a cross validation method
# the cv method will automatically figure out what to do.
scores = cross_val_score(dt_pipe, X_train, y_train, cv=4, scoring = 'accuracy')
print(f"the mean score is: {scores.mean()}")
print("The scores were:", scores)

the mean score is: 0.9673453199365416
The scores were: [0.96774194 0.96721311 0.95081967 0.98360656]


In [ ]:
top_tags = posts_proto['tags'].value_counts().head(50)